In [1]:
from pyspark import SparkContext

In [2]:
sc = SparkContext()

In [3]:
%%writefile example.txt
first line
seconds line
third line
fourth line

Writing example.txt


In [4]:
textFile = sc.textFile('example.txt')

In [5]:
textFile.count()

4

In [6]:
textFile.first()

'first line'

In [7]:
secfind = textFile.filter(lambda line: 'second' in line)

In [9]:
secfind

PythonRDD[4] at RDD at PythonRDD.scala:48

In [10]:
secfind.collect()

['seconds line']

In [11]:
secfind.count()

1

In [12]:
%%writefile example2.txt
first line
second line
third line
fourth line

Writing example2.txt


In [14]:
text_rdd = sc.textFile('example2.txt')

In [16]:
words = text_rdd.map(lambda line: line.split())

In [18]:
words.collect()

[['first', 'line'], ['second', 'line'], ['third', 'line'], ['fourth', 'line']]

In [19]:
text_rdd.collect()

['first line', 'second line', 'third line', 'fourth line']

In [22]:
text_rdd.flatMap(lambda line: line.split()).collect()

['first', 'line', 'second', 'line', 'third', 'line', 'fourth', 'line']

In [23]:
%%writefile services.txt
#EventId    Timestamp    Customer   State    ServiceID    Amount
201       10/13/2017      100       NY       131          100.00
204       10/18/2017      700       TX       129          450.00
202       10/15/2017      203       CA       121          200.00
206       10/19/2017      202       CA       131          500.00
203       10/17/2017      101       NY       173          750.00
205       10/19/2017      202       TX       121          200.00

Writing services.txt


In [24]:
services = sc.textFile('services.txt')

In [25]:
services.take(2)

['#EventId    Timestamp    Customer   State    ServiceID    Amount',
 '201       10/13/2017      100       NY       131          100.00']

In [26]:
services.map(lambda line: line.split()).take(3)

[['#EventId', 'Timestamp', 'Customer', 'State', 'ServiceID', 'Amount'],
 ['201', '10/13/2017', '100', 'NY', '131', '100.00'],
 ['204', '10/18/2017', '700', 'TX', '129', '450.00']]

In [28]:
clean = services.map(lambda line: line[1:] if line[0] == '#' else line)

In [29]:
clean = clean.map(lambda line: line.split())

In [30]:
clean.collect()

[['EventId', 'Timestamp', 'Customer', 'State', 'ServiceID', 'Amount'],
 ['201', '10/13/2017', '100', 'NY', '131', '100.00'],
 ['204', '10/18/2017', '700', 'TX', '129', '450.00'],
 ['202', '10/15/2017', '203', 'CA', '121', '200.00'],
 ['206', '10/19/2017', '202', 'CA', '131', '500.00'],
 ['203', '10/17/2017', '101', 'NY', '173', '750.00'],
 ['205', '10/19/2017', '202', 'TX', '121', '200.00']]

In [33]:
pairs = clean.map(lambda lst: (lst[3],lst[-1]))

In [39]:
rekey = pairs.reduceByKey(lambda amt1,amt2:float(amt1)+float(amt2))

In [40]:
rekey.collect()

[('State', 'Amount'), ('NY', 850.0), ('TX', 650.0), ('CA', 700.0)]

In [41]:
clean.collect()

[['EventId', 'Timestamp', 'Customer', 'State', 'ServiceID', 'Amount'],
 ['201', '10/13/2017', '100', 'NY', '131', '100.00'],
 ['204', '10/18/2017', '700', 'TX', '129', '450.00'],
 ['202', '10/15/2017', '203', 'CA', '121', '200.00'],
 ['206', '10/19/2017', '202', 'CA', '131', '500.00'],
 ['203', '10/17/2017', '101', 'NY', '173', '750.00'],
 ['205', '10/19/2017', '202', 'TX', '121', '200.00']]

In [42]:
step1 = clean.map(lambda lst: (lst[3],lst[-1]))

In [43]:
step2 = step1.reduceByKey(lambda amt1, amt2: float(amt1)+float(amt2))

In [44]:
step3 = step2.filter(lambda x: not x[0] == 'State')

In [45]:
step4 = step3.sortBy(lambda stAmount: stAmount[1],ascending=False)

In [46]:
step4.collect()

[('NY', 850.0), ('CA', 700.0), ('TX', 650.0)]

In [47]:
x = ['ID', 'State', 'Amount']

In [48]:
def func1(lst):
    return lst[-1]

In [49]:
def func2(id_st_amt):
    #unpack
    (ID,st,amt) = id_st_amt
    return amt

In [51]:
func1(x)

'Amount'

In [52]:
func2(x)

'Amount'